In [186]:
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler, normalize

In [187]:
import warnings
warnings.filterwarnings('ignore')

In [188]:
macro = pd.read_csv('../data/macro_with_shift.csv')

In [189]:
macro

,date,target,DTB3,MCUMFN,CPIAUCSL,GEPU_current,IPG211111CN,Killian,IRLTLT01USM156N,M2SL,Close,CFNAI,UNRATE,Imports,Production,Stocks,target_date
0,1997-01-02,25.55,5.033810,82.0867,159.400,76.906476,98.3693,-4.346762,6.58,3834.6,786.16,0.26,5.3,7218.20,6489.4,838890.8,1997-02-03
1,1997-01-03,25.55,5.033810,82.0867,159.400,76.906476,98.3693,-4.346762,6.58,3834.6,786.16,0.26,5.3,7218.20,6489.4,838890.8,1997-02-04
2,1997-01-06,26.25,5.033810,82.0867,159.400,76.906476,98.3693,-4.346762,6.58,3834.6,786.16,0.26,5.3,7218.20,6489.4,838890.8,1997-02-05
3,1997-01-07,26.25,5.033810,82.0867,159.400,76.906476,98.3693,-4.346762,6.58,3834.6,786.16,0.26,5.3,7218.20,6489.4,838890.8,1997-02-06
4,1997-01-08,26.55,5.033810,82.0867,159.400,76.906476,98.3693,-4.346762,6.58,3834.6,786.16,0.26,5.3,7218.20,6489.4,838890.8,1997-02-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6081,2021-03-25,58.47,0.031304,76.3791,265.028,212.409145,165.8403,24.093239,1.61,19853.7,3972.89,2.22,6.0,5686.25,10975.0,1138710.0,2021-04-27
6082,2021-03-26,60.93,0.031304,76.3791,265.028,212.409145,165.8403,24.093239,1.61,19853.7,3972.89,2.22,6.0,5686.25,10975.0,1138710.0,2021-04-28
6083,2021-03-29,61.49,0.031304,76.3791,265.028,212.409145,165.8403,24.093239,1.61,19853.7,3972.89,2.22,6.0,5686.25,10975.0,1138710.0,2021-04-29
6084,2021-03-30,60.55,0.031304,76.3791,265.028,212.409145,165.8403,24.093239,1.61,19853.7,3972.89,2.22,6.0,5686.25,10975.0,1138710.0,2021-04-30


In [190]:
tech = pd.read_csv('../data/technical_dataset.csv')

In [191]:
macro_date = macro['date']
y = macro['target']
macro.drop(columns=['target_date', 'target', 'date'], inplace=True)

In [192]:
tech_date = tech['date']
tech.drop(columns=['date'], inplace=True)

# Fitting and validation, nothing different from the default parameters

In [193]:
data = pd.concat([macro, tech], axis=1)

In [9]:
data

,DTB3,MCUMFN,CPIAUCSL,GEPU_current,IPG211111CN,Killian,IRLTLT01USM156N,M2SL,Close,CFNAI,...,MOM(3),MOM(6),MOM(9),MOM(12),OBV1_9,OBV2_9,OBV3_9,OBV1_12,OBV2_12,OBV3_12
0,5.033810,82.0867,159.400,76.906476,98.3693,-4.346762,6.58,3834.6,786.16,0.26,...,1,0,1,1,1,1,1,1,1,1
1,5.033810,82.0867,159.400,76.906476,98.3693,-4.346762,6.58,3834.6,786.16,0.26,...,1,0,0,1,1,1,1,1,1,1
2,5.033810,82.0867,159.400,76.906476,98.3693,-4.346762,6.58,3834.6,786.16,0.26,...,1,1,0,0,1,1,1,1,1,1
3,5.033810,82.0867,159.400,76.906476,98.3693,-4.346762,6.58,3834.6,786.16,0.26,...,1,1,1,1,1,1,1,1,1,1
4,5.033810,82.0867,159.400,76.906476,98.3693,-4.346762,6.58,3834.6,786.16,0.26,...,1,1,0,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6081,0.031304,76.3791,265.028,212.409145,165.8403,24.093239,1.61,19853.7,3972.89,2.22,...,0,0,0,0,1,0,0,0,0,0
6082,0.031304,76.3791,265.028,212.409145,165.8403,24.093239,1.61,19853.7,3972.89,2.22,...,0,0,0,0,0,1,0,0,0,0
6083,0.031304,76.3791,265.028,212.409145,165.8403,24.093239,1.61,19853.7,3972.89,2.22,...,1,1,0,0,1,1,1,1,0,0
6084,0.031304,76.3791,265.028,212.409145,165.8403,24.093239,1.61,19853.7,3972.89,2.22,...,1,1,0,0,1,1,1,1,1,1


In [171]:
x_train = data.iloc[:5813]
y_train = y.iloc[:5813]
x_test = data.iloc[5813:]
y_test = y[5813:]

In [172]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train_sc = scaler.transform(x_train)
x_test_sc = scaler.transform(x_test)

In [173]:
x_train_norm = normalize(x_train)
x_test_norm = normalize(x_test)

In [174]:
model = ElasticNet()

In [176]:
model.fit(x_train_sc, y_train)
pred = model.predict(x_test_sc)
print(r2_score(y_test, pred))
print(mean_squared_error(y_test, pred))

-14.518045837146628
2607.333273349191


# Fitting with feature selection, ElasticNet selects features for ElasticNet, Lasso selects features for Lasso, we look at non-zero coefficients (we first fit and look at coefficients, and then we fit using features with non-zero coefficients)

In [177]:
coef = model.coef_
cols = x_train.columns
zipped = list(zip(cols, coef))

In [178]:
res = sorted(zipped, key = lambda x: x[1])

In [179]:
res

[('IRLTLT01USM156N', -3.528371564225259),
 ('Production', -2.348395260168305),
 ('DTB3', -1.7165386237119205),
 ('IPG211111CN', -1.6690618291080455),
 ('GEPU_current', -0.823426320415812),
 ('CFNAI', -0.6086975723398889),
 ('Imports', 0.0),
 ('MA1_9', -0.0),
 ('MA2_9', -0.0),
 ('MA3_9', -0.0),
 ('MA1_12', -0.0),
 ('MA2_12', -0.0),
 ('MA3_12', -0.0),
 ('MOM(1)', 0.0),
 ('MOM(2)', 0.0),
 ('MOM(3)', 0.0),
 ('MOM(6)', -0.0),
 ('MOM(9)', 0.0),
 ('OBV1_9', 0.0),
 ('OBV2_9', 0.0),
 ('OBV3_9', 0.0),
 ('OBV2_12', 0.241557631059662),
 ('MOM(12)', 0.31378429165857563),
 ('OBV3_12', 0.38451280448170727),
 ('OBV1_12', 0.395554068753654),
 ('Close', 0.5066181814708504),
 ('MCUMFN', 1.3021935493878267),
 ('Stocks', 2.020321549838751),
 ('M2SL', 2.9393157465263027),
 ('Killian', 5.455012960398992),
 ('UNRATE', 5.997554567258016),
 ('CPIAUCSL', 7.806335285205878)]

In [180]:
features = []
for i in res:
    if i[1] != 0.0:
        features.append(i[0])

In [181]:
x_train = data[features].iloc[:5813]
y_train = y.iloc[:5813]
x_test = data[features].iloc[5813:]
y_test = y.iloc[5813:]

In [182]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train_sc = scaler.transform(x_train)
x_test_sc = scaler.transform(x_test)

In [183]:
x_train_norm = normalize(x_train)
x_test_norm = normalize(x_test)

In [185]:
model.fit(x_train_norm, y_train)
pred = model.predict(x_test_norm)
print(r2_score(y_test, pred))
print(mean_squared_error(y_test, pred))

-1.3347610178439528
392.28522399199835


# Fitting when two sets of features(macro and technical) are used separately

In [104]:
data = tech

In [105]:
x_train = data.iloc[:5813]
y_train = y.iloc[:5813]
x_test = data.iloc[5813:]
y_test = y[5813:]

In [106]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train_sc = scaler.transform(x_train)
x_test_sc = scaler.transform(x_test)

In [107]:
x_train_norm = normalize(x_train)
x_test_norm = normalize(x_test)

In [113]:
model = ElasticNet()

In [114]:
model.fit(x_train_sc, y_train)
pred = model.predict(x_test_sc)
print(r2_score(y_test, pred))
print(mean_squared_error(y_test, pred))

-1.3678622912267882
397.8468811998085


# Fitting with economic constraints. An economic constraint that a rational investor will rule out a negative stock return forecast and therefore set the forecast to zero whenever it is negative

In [ ]:
data = pd.concat([macro, tech], axis=1)

In [194]:
data['return'] = -0.035

In [195]:
data['return'].iloc[1:] = y.pct_change().iloc[1:]

In [196]:
y = data['return']
data.drop(columns = ['return'], inplace=True)

In [197]:
x_train = data.iloc[:5813]
y_train = y.iloc[:5813]
x_test = data.iloc[5813:]
y_test = y[5813:]

In [198]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train_sc = scaler.transform(x_train)
x_test_sc = scaler.transform(x_test)

In [199]:
x_train_norm = normalize(x_train)
x_test_norm = normalize(x_test)

In [200]:
model = ElasticNet()

In [204]:
model.fit(x_train, y_train)
pred = model.predict(x_test)
pred = [num if num > 0 else 0 for num in pred]
print(r2_score(y_test, pred))
print(mean_squared_error(y_test, pred))

-0.0022608407974593625
0.044048728905818874
